### Approach to the solution

##### 1. Which ML/DL model architecture you used and why?
- I am using webscrapping by utilising the yahoo's search engine to get the most relevant articles for the given query.
- I have used LDA i.e. <b>Latent Dirichlet Model</b> to train my model primarily because it is currently one of the most popular topic modelling technique.
- There are other techniques as well such Term Frequency and Inverse Document Frequency, NonNegative Matrix Factorization techniques which I haven't used for now.

##### 2. How would you ensure the scalability of your solution?
- For the top 5 articles, we mostly deal with less dataset. Incase we start dealing with more and more articles, our dataset increases and hence according to me the Cloud solution is the best. We can use multi-GPU power of DataFlow pipeline to create our compressed dataset in the form of tfrecords and then deploy over AI Platform. This Multi-GPU approach makes this solution highly scalable.
- And if we talk about the scalability of the number of articles then we have to make sure that we are able to navigate to the next pages(eg: yahoo's next pages in our case). Once we achieve this we become fairly confident that even on increasing the number of articles we don't get limited by the articles that we can access.

##### 3. Is there a need for any dataset? If yes then how much data is sufficient to train the model in order to get the required results?
- Yes. We are scrapping the top query relevant articles from the web. So, the result of this webscrapping would be the multiple web articles out of which we have to ensure few parameters as follows:
    - articles text size
    - number of unique words in each article
    - total vocabulary size from all the articles
    - Permission to scrap the website
- Once we make sure that we satisfy the above parameters then we are good to select our top five required articles.
- Dataset with unqiue words of more than 1000 should be suffiecient to get the required results.

##### 4. Is there a need to create manual datasets, if yes then what parameters and sample size did you consider to create a dataset? 
- No.
- If we use potentially strong search engine then we will always get enough number of articles.
- Even if the articles on the first page are not sufficient, still we can move over to the next page to webscrap further articles.

##### 5. Is your model and dataset generalized enough for different domains of the use cases, How?
- Yes.
- As discussed in the point two, I have considered the parameters sensitive for the possible test cases.
- We can further improve it by getting accessible to the articles over the next pages. The reason I haven't implemented this feature is because we are dealing with only 5 articles so I didn't find the need for it in our usecase.
- Also the model I have chosed is LDA which is currently the most popular in topic/keywords modelling.

##### 6. How would you train, test and deploy your model to production? 
- One of the production effiecint technique which I know is by using GCP's DataFlow Pipeline and AI Platform's tf.estimator API both of which allows us the use of multiple GPUs.
- Using DataFlow pipeline we can have even terabytes of data in our articles which will be converted into tfrecords and get stored on the cloud storage.
- Once the tfrecords are obtained, we can use AI Platform which allows us to perform 3 operations on the dataset anytime.
- i.e.    i) training ii) evaluation iii) prediction
- On AI Platform, we can deploy our model and then our code will run in the prediction mode as per the tf estimator API.

##### 7. How would you perform hyperparameter tuning on your model to improve accuracy?
- I have provided the solution with the visualization, I have observed that the topics don't get overlap mostly when I set 3 topics.
- Possible reason for this could be a less dataset that is getting scrapped becasue we are dealling with only top 5 articles.
- Had been the case that we were using more articles, we would have got the freedom to get the results from even more topics.
- Hyperparameters : num of topics, number of unqiue words required, article.text length
- Even we can go for the other search engines as well which will just require more work right from getting the html elements.

##### 8. Anything else you want to let us know about your approach.
- The current approach looks fairly appealing to me because the differences in the results are direclty visible to me.
- However there is certain overlaps in the topics as we go on increasing the number of topic becasue we are only using the dataset from 5 articles.
- Still this LDA approach can be utilsed for even one topic which will atleast provide us with the most relevant keywords. However this model is fairly working amazing even with 3 topics and even sometimes upto 5 even for such smaller dataset.

##### <hr>
### Model/Code on GitHub repo or Colab Notebook with the necessary documentation describing the model functioning.
- Code is pushed on my github repo. It can be found on this url : https://github.com/vishalw-iitk/ML_new/blob/master/CareerNinja/vishal_w.ipynb

##### <hr>
### Only Approach to generate different types of questions (short answer type, MCQs, true/false, fill in the blanks, long answer type, etc.) for that same article.
- We can generate questions using <b>allennlp</b> for all the articles
- Or this is one of the theoritical approach I know currently:
    - Identify keywords from the text and use them as answers to the questions.
    - Replace the answer from the sentence with blank space and use it as the base for the question.
    - Transform the sentence with a blank space for answer to a more question-like sentence.
    - Generate distractors, words that are similar to the answer, as incorrect answers.

In [1]:
import re # To use regex operations
from bs4 import BeautifulSoup #To parse response text using the html parser
import requests #To make request from the web

In [2]:
import nltk
nltk.download('stopwords', quiet=True) #stopwords
nltk.download('wordnet', quiet=True) #
from newspaper import Article #To read the content of a webpage

In [3]:
import gensim #Gensim is an open-source library for unsupervised topic modeling and natural language processing
from gensim.models.ldamulticore import LdaMulticore # for multipreprocessing
from gensim import corpora, models #
from gensim.corpora import Dictionary
import pyLDAvis.gensim #for visualization

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer


# from itertools import chain

In [4]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')

c:\users\vishal waghmare\3d objects\vscode_codes\github\pandas\careerninja\careerninja\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# We are using Yahoo Search Engine for WebScrapping
template = 'https://in.search.yahoo.com/search;_ylt=AwrwIQa9FFJgmlQA7hq7HAx.;_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3BpdnM-?p={}'

In [37]:
query = input("Enter your main query(eg: Earphones in the market) : ")
url = template.format(query)

Enter your main query(eg: Earphones in the market) : Earphones in the market


In [38]:
# To make the get request, we first set the request headers
headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

In [39]:
response = requests.get(url, headers=headers)

In [40]:
soup = BeautifulSoup(response.text, 'html.parser')

In [41]:
# cards = soup.find_all('div', 'NewsArticle')
cards = soup.find_all('div', 'algo')

In [42]:
# All the articles result currently in the html form
len(cards)

9

In [43]:
#Function to get the headline of the article and its url
def get_article(card):
    '''Extract article information from the raw html'''
#     headline = card.find('h4', 's-title').text
    headline = card.find('h3', 'title').text
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    article = [headline, clean_link]
    return article

In [44]:
# Storing the headline and url for every article on the first page in the articles list
articles = []
links = set()
for card in cards:
    article = get_article(card)
    link = article[-1]
    if not link in links:
        links.add(link)
        articles.append(article)

In [29]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [30]:
class top_article:
    def __init__(self, article_name, article_link):
        '''To set article name and article link for every article'''
        self.article_name = article_name
        self.article_link = article_link

    def get_uncleaned_webcontent(self):
        '''To get the cleaned web article from all the web articles'''
        
        article = Article(self.article_link)
        
        try :
            article.download()
            article.parse()
            nltk.download('punkt', quiet = True)
            article.nlp()
            
            if len(article.text) > 100:
                '''Taking only the articles with total text length greater than 100'''
                self.article_content = article.text
            else:
                self.article_content = "It is invalid. Go for next"
            
        except:
            '''Exception if the article does not have permission to get scrapped'''
            self.article_content = "It is invalid. Go for next"
        
        
    def clean(self):
        '''
        Splitting the data
        Removing the punctuations
        Lemmatization : making sure that words like spectacle/spectacles are considered same
        '''
        stop_free = ' '.join([word for word in self.article_content.lower().split() if word not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
        self.article_cleaned =  normalized.split()

In [45]:
def make_dictionary(bag_of_words):
    '''Make a single dictionary of the words from top 5 articles'''
    dictionary = corpora.Dictionary(bag_of_words)
    return dictionary

def make_doc_term_matrix(dictionary, bag_of_words):
    '''
    A matrix which stores the occurence and frequency of each word from the bag of words
    As we have 5 articles, this matrix will have 5 rows
    '''
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in bag_of_words]
    return doc_term_matrix

def build_lda_model(dictionary, doc_term_matrix, lda, num_topics):
    '''Training the model by passing the hyperparameters'''
    lda_model = lda(doc_term_matrix, num_topics = num_topics, id2word=dictionary, passes=50, minimum_probability=0)
    return lda_model

def print_topic_clusters(lda_model, num_topics):
    '''
    Print the clusters of selected number of topics
    Each cluster shows the relevant keywords associated with that cluster using which we can determine the topic of that cluster
    '''
    print('\033[1m')
    print("\nKeywords/Topics/Tags for the articles\n")
    print('\033[0m')
    keywords_and_probs = lda_model.print_topics(num_topics=num_topics)
    keywords_only = []
    for i in keywords_and_probs:
        '''Original output contains the probability along with the keywords. Here we are trying to extract only the keywords'''
        keywords_only.append(re.findall(r'"(.*?)"', i[1]))
    for i, cluster_keywords in enumerate(keywords_only):
        '''We print the cluster number and the keywords associated with that cluster'''
        print("Cluster",i+1,cluster_keywords)

def build_lda_display(lda_model, doc_term_matrix, dictionary):
    '''Here we build the lda_display object which helps us to visualize our results'''
    lda_display = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False, mds='mmds')
    return lda_display

In [46]:
top_articles = []
for article in articles:
    '''Getting all articles obtained from the search engine. This articles are obtained on the 1st page'''
    top_articles.append(top_article(article[0], article[1]))

# Initializing the model object
lda = gensim.models.ldamodel.LdaModel


# User input for the number of topics would we like to achieve
num_topics = int(input("Enter the number of topics we want to get(recommended less than 5)(say 3) :  "))
print("\nPlease wait...")


count = 0
valid_top_articles = []
for num, article in enumerate(top_articles):
    if count >= 5:
        # Inorder to get the top 5 articles
        break
    article.get_uncleaned_webcontent() #Getting the data for every top 5 article
    if article.article_content == "It is invalid. Go for next":
        continue
    count = count + 1
    article.clean() #Cleaning the the data of every top 5 articles
    valid_top_articles.append(article)

print("\nTop 5 articles are :")
bag_of_words = []
for num, article in enumerate(valid_top_articles):
    '''Printing the url for top 5 articles and creating a bag of words from all of them'''
    print(str(num+1)+".", article.article_link)
    bag_of_words.append(article.article_cleaned)

dictionary = make_dictionary(bag_of_words) #Making a dcitionary from the bag of words
doc_term_matrix = make_doc_term_matrix(dictionary, bag_of_words) #creating a matrix

print("\nBuilding the clusters of topics....")
lda_model = build_lda_model(dictionary, doc_term_matrix, lda, num_topics) #training the model

print_topic_clusters(lda_model, num_topics) #printing the cluster results
lda_display = build_lda_display(lda_model, doc_term_matrix, dictionary) #building the object capable to show visualization of the results

Enter the number of topics we want to get(recommended less than 5)(say 3) :  3

Please wait...

Top 5 articles are :
1. https://www.grandviewresearch.com/industry-analysis/earphone-and-headphone-market
2. https://www.gminsights.com/industry-analysis/earphone-and-headphone-market
3. https://www.transparencymarketresearch.com/earphone-market.html
4. https://www.marketwatch.com/press-release/global-hi-fi-earphones-market-2020-applications-market-size-according-to-a-specific-product-sales-and-revenue-by-region-2025-2021-03-17
5. https://www.mordorintelligence.com/industry-reports/earphones-and-headphones-market

Building the clusters of topics....




Keywords/Topics/Tags for the articles


Cluster 1 ['earphone', 'headphone', 'market', 'wireless', 'corporation', 'inc', 'audio', 'technology', '2020', 'consumer']
Cluster 2 ['market', 'report', 'earphone', 'research', 'business', 'study', 'global', 'customer', 'key', 'analysis']
Cluster 3 ['done', 'nicca', 'brian', 'the', 'vp', 'excellent', 'moore', 'usa', 'inc', 'quality']


In [47]:
pyLDAvis.display(lda_display) #Visualization of the results(works on local jupyter notebook).   It may not be visible over github.

In [48]:
# Assigning the topics to the documents in the corpus
lda_corpus = lda_model[doc_term_matrix]
lda_corpus

In [49]:
# Printing the probability score for topics for our top 5 articles
for i, article in enumerate(lda_corpus):
    print("\n\nFor article",i+1)
    print("probability score for: \n")
    for j, topic in enumerate(article):
        print("topic",j+1,":",topic[1])
    print("-----------------------------------")



For article 1
probability score for: 

topic 1 : 0.06905838
topic 2 : 0.9044903
topic 3 : 0.026451332
-----------------------------------


For article 2
probability score for: 

topic 1 : 0.99939144
topic 2 : 0.00031406892
topic 3 : 0.00029452212
-----------------------------------


For article 3
probability score for: 

topic 1 : 0.0003692049
topic 2 : 0.9992921
topic 3 : 0.00033874266
-----------------------------------


For article 4
probability score for: 

topic 1 : 0.00065642956
topic 2 : 0.99872684
topic 3 : 0.0006167119
-----------------------------------


For article 5
probability score for: 

topic 1 : 0.1932703
topic 2 : 0.8028708
topic 3 : 0.003858884
-----------------------------------
